In [1]:
import numpy as np
import torch 
import matplotlib.pyplot as plt
import torch
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from PIL import Image

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Operators:
    @staticmethod
    def PauliX():
        return torch.tensor([[0, 1], [1, 0]], dtype=torch.complex64, device=device)

    @staticmethod
    def PauliY():
        return torch.tensor([[0, -1j], [1j, 0]], dtype=torch.complex64, device=device)

    @staticmethod
    def PauliZ():
        return torch.tensor([[1, 0], [0, -1]], dtype=torch.complex64, device=device)

    @staticmethod
    def Rx(theta):
        return torch.cos(theta / 2) * torch.eye(2, dtype=torch.complex64, device=device) - \
               1j * torch.sin(theta / 2) * Operators.PauliX()

    @staticmethod
    def Ry(theta):
        return torch.cos(theta / 2) * torch.eye(2, dtype=torch.complex64, device=device) - \
               1j * torch.sin(theta / 2) * Operators.PauliY()

    @staticmethod
    def Rz(theta):
        return torch.cos(theta / 2) * torch.eye(2, dtype=torch.complex64, device=device) - \
               1j * torch.sin(theta / 2) * Operators.PauliZ()

    @staticmethod
    def CNOT():
        I = torch.eye(2, dtype=torch.complex64, device=device)
        zero_proj = torch.tensor([[1, 0], [0, 0]], dtype=torch.complex64, device=device)
        one_proj = torch.tensor([[0, 0], [0, 1]], dtype=torch.complex64, device=device)
        return torch.kron(zero_proj, I) + torch.kron(one_proj, Operators.PauliX())

class utils:
    def apply_one_site(site, state, op, L=32):
        state = state.reshape(2**(site), 2, 2**(L-site-1))
        state = torch.tensordot(op, state, dims=([1], [1]))
        state = state.permute(1, 0, 2).contiguous()
        return state.reshape(-1,)

    def apply_two_site(sites, state, op, L=32):
        state = state.reshape(2**sites[0], 2, 2**(sites[1]-sites[0]-1), 2, 2**(L-sites[1]-1))
        op = op.view(2, 2, 2, 2)
        state = torch.tensordot(op, state, dims=([2, 3], [1, 3]))
        state = state.permute(2, 0, 3, 1, 4).contiguous()
        return state.reshape(-1,)

    def measure_expectation(state, op, site, L=32):
        state = state.view(2**(site), 2, 2**(L-site-1))
        measured_state = torch.tensordot(op, state, dims=([1], [1]))
        expectation_value = torch.vdot(state.view(-1), measured_state.view(-1)).real
        return expectation_value

def layer(params, state, L=32):
    for i in range(L):
        Ry_gate = Operators.Ry(params[i])
        state = utils.apply_one_site(i, state, Ry_gate, L)
    for i in range(L-1):
        state = utils.apply_two_site([i, i+1], state, Operators.CNOT(), L)
    return state

def circuit(params, state, L=32):
    n_layers = params.shape[0]
    for i in range(n_layers):
        state = layer(params[i], state, L=L)
    return utils.measure_expectation(state, Operators.PauliZ(), site=7, L=L)

# Load and preprocess MNIST dataset
from sklearn.datasets import fetch_openml
from PIL import Image

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
images, labels = mnist.data, mnist.target.astype(int)

indices_2 = np.where(labels == 2)[0][:2000]
indices_3 = np.where(labels == 3)[0][:2000]
selected_indices = np.concatenate((indices_2, indices_3))
selected_images = images[selected_indices]
selected_labels = labels[selected_indices]

selected_labels = np.where(selected_labels == 2, -1, 1)

def preprocess_image(image):
    image = image.reshape(28, 28)
    image_resized = Image.fromarray(np.uint8(image))
    image_resized = image_resized.resize((16, 16), Image.Resampling.LANCZOS)
    image_resized = np.array(image_resized).reshape(-1,)
    return image_resized / np.linalg.norm(image_resized)

selected_images = np.array([preprocess_image(img) for img in selected_images])

X_data = torch.tensor(selected_images, dtype=torch.float32, device=device)
Y_data = torch.tensor(selected_labels, dtype=torch.float32, device=device)

train_size = int(0.8 * len(X_data))
X_train, X_test = X_data[:train_size], X_data[train_size:]
Y_train, Y_test = Y_data[:train_size], Y_data[train_size:]

batch_size = 50
train_loader = torch.utils.data.DataLoader(list(zip(X_train, Y_train)), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(list(zip(X_test, Y_test)), batch_size=batch_size, shuffle=False)

print("Data preprocessing complete! Ready for training.")

class QuantumCircuitModel(nn.Module):
    def __init__(self, n_layers=32, n_qubits=8):
        super().__init__()
        self.n_layers = n_layers
        self.n_qubits = n_qubits
        self.params = nn.Parameter(torch.rand(n_layers, n_qubits) * torch.pi*0.01)  


    def forward(self, x):
        batch_size, input_dim = x.shape  
        assert input_dim == 2**self.n_qubits, f"Expected input_dim={2**self.n_qubits}, but got {input_dim}"
    

        state = x.to(dtype=torch.complex64,device=x.device) 

        results = torch.vmap(lambda s: circuit(self.params, s, L=self.n_qubits))(state)
    
        return results 


model = QuantumCircuitModel().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

def convert_labels(y):
    return (y + 1) / 2  

epochs = 50

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_train, total_train = 0, 0 

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = convert_labels(labels)  

        optimizer.zero_grad()
        outputs = model(images)  

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predictions = torch.sigmoid(outputs) > 0.5
        correct_train += (predictions == labels).sum().item()
        total_train += labels.size(0)
    
    train_accuracy = correct_train / total_train
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.4f},Train Accuracy: {train_accuracy * 100:.2f}%")

model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels = convert_labels(labels)  

        outputs = model(images)
        predictions = torch.sigmoid(outputs) > 0.5  
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Data preprocessing complete! Ready for training.
Epoch [1/50], Loss: 0.6043,Train Accuracy: 86.81%
Epoch [2/50], Loss: 0.5646,Train Accuracy: 93.66%
Epoch [3/50], Loss: 0.5578,Train Accuracy: 92.50%
Epoch [4/50], Loss: 0.5525,Train Accuracy: 93.53%
Epoch [5/50], Loss: 0.5491,Train Accuracy: 94.38%
Epoch [6/50], Loss: 0.5472,Train Accuracy: 94.06%
Epoch [7/50], Loss: 0.5435,Train Accuracy: 94.88%
Epoch [8/50], Loss: 0.5422,Train Accuracy: 94.75%
Epoch [9/50], Loss: 0.5410,Train Accuracy: 94.66%
Epoch [10/50], Loss: 0.5406,Train Accuracy: 94.59%
Epoch [11/50], Loss: 0.5405,Train Accuracy: 93.72%
Epoch [12/50], Loss: 0.5393,Train Accuracy: 94.66%
Epoch [13/50], Loss: 0.5383,Train Accuracy: 94.78%
Epoch [14/50], Loss: 0.5379,Train Accuracy: 94.84%
Epoch [15/50], Loss: 0.5371,Train Accuracy: 94.88%
Epoch [16/50], Loss: 0.5371,Train Accuracy: 94.72%
Epoch [17/50], Loss: 0.5364,Train Accuracy: 95.03%
Epoch [18/50], Loss: 0.5366,Train Accuracy: 94.41%
Epoch [19/50], Loss: 0.5366,Train Accuracy

In [2]:
def circuit2(params, state, L=8):
    n_layers = params.shape[0]

    for i in range(n_layers):
        state = layer(params[i], state, L=L)
    probabilities = torch.abs(state) ** 2  

    probabilities = probabilities / torch.sum(probabilities)

    return probabilities 

trained_params = model.params.detach()

with torch.no_grad():
    quantum_states = torch.zeros((X_data.shape[0], 2**8), dtype=torch.complex64, device=device)
    quantum_states[:, :2**8] = X_data
    quantum_states /= torch.norm(quantum_states, dim=1, keepdim=True)  # Normalize batch-wise

    batch_probabilities = torch.vmap(lambda s: circuit2(trained_params, s, L=8))(quantum_states)

    mask_class_minus1 = (Y_data == -1)
    mask_class_plus1 = (Y_data == 1)

    state_list_class_minus1 = batch_probabilities[mask_class_minus1]
    state_list_class_plus1 = batch_probabilities[mask_class_plus1]


assert state_list_class_minus1.numel() > 0, "Error: No states found for class -1!"
assert state_list_class_plus1.numel() > 0, "Error: No states found for class +1!"


state_minus = torch.mean(state_list_class_minus1, dim=0) 
state_plus = torch.mean(state_list_class_plus1, dim=0)   

rho_0 = state_minus[:, None] * state_minus.conj()[None, :] 
rho_1 = state_plus[:, None] * state_plus.conj()[None, :]

rho_diff = rho_0 - rho_1
eigenvalues = torch.linalg.eigvalsh(rho_diff) 
trace_distance = 0.5 * torch.sum(torch.abs(eigenvalues))  

helstrom_bound = 0.5 + 0.5 * trace_distance.item()
print(f"Helstrom Bound: {helstrom_bound:.4f}")


Helstrom Bound: 0.5031


In [3]:
Mnist2 = []
Mnist3 = []
for i in range(0,25):
    test = X_data[i].cpu()
    test /= torch.norm(test)
    rho = torch.outer(test,test.conj())
    Mnist2.append(rho)
for i in range(2000,2025):
    test = X_data[i].cpu()
    test /= torch.norm(test)
    rho = torch.outer(test,test.conj())
    Mnist3.append(rho)


In [4]:
rho2 = torch.mean(torch.stack(Mnist2),dim=0)
rho3 = torch.mean(torch.stack(Mnist3),dim=0)
rho_diff = rho2-rho3
eigenvalues = torch.linalg.eigvalsh(rho_diff) 
trace_distance = 0.5 * torch.sum(torch.abs(eigenvalues)) 
helstrom_bound = 0.5 + 0.5 * trace_distance.item()
print(f"Helstrom Bound: {helstrom_bound:.4f}")

Helstrom Bound: 0.8379
